## Object-Relational Mapper exercise


---

Using relational databases directly in programming languages other than SQL can be tedious and error-prone. One of the common approaches is to use an ORM (object-relational mapper) for that.


One of the popular ORMs for Python is SQLAlchemy.

https://www.sqlalchemy.org/

---

For the purpose of the exercise, I will create an in-memory SQLite database:

In [47]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from decimal import Decimal
import warnings

from sqlalchemy.orm import relationship


warnings.filterwarnings("ignore")

engine = sa.create_engine('sqlite:///:memory:')
Base = declarative_base()
session = sa.orm.sessionmaker(bind=engine)()


Now, I will create three tables for the exercise:

In [48]:

class OrderItem(Base):
    __tablename__ = 'order'

    id = sa.Column('id', sa.Integer, primary_key=True)
    product_id = sa.Column('product_id', sa.Integer, sa.ForeignKey('product.id'))
    user_id = sa.Column('user_id', sa.Integer, sa.ForeignKey('user.id'))
    qty = sa.Column('qty', sa.Integer)

class Product(Base):
    __tablename__ = 'product'

    id = sa.Column('id', sa.Integer, primary_key=True)
    name = sa.Column('name', sa.Text)
    price = sa.Column('price', sa.Numeric(14,2))

class User(Base):
    __tablename__ = 'user'

    id = sa.Column('id', sa.Integer, primary_key=True)
    first_name = sa.Column('first_name', sa.Text)
    last_name = sa.Column('last_name', sa.Text)

Base.metadata.create_all(engine)

engine.echo = True



And populate their content with records:

In [49]:

session.add(User(id=42, first_name='John', last_name='Cleese'))
session.add(User(id=43, first_name='Eric', last_name='Idle'))
session.add(User(id=44, first_name='Terry', last_name='Jones'))
session.add(User(id=13, first_name='Graham', last_name='Chapman'))
session.add(Product(id=1, price=Decimal('2.10'), name='Rabbit'))
session.add(Product(id=2, price=Decimal('5.10'), name='Parrot'))
session.add(OrderItem(product_id=1, user_id=42, qty=9))
session.add(OrderItem(product_id=1, user_id=42, qty=2))
session.add(OrderItem(product_id=1, user_id=13, qty=2))
session.add(OrderItem(product_id=2, user_id=13, qty=3))
session.add(OrderItem(product_id=2, user_id=13, qty=20))
session.commit()

2021-07-26 13:16:05,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-26 13:16:05,584 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-26 13:16:05,584 INFO sqlalchemy.engine.Engine [generated in 0.00054s] (1, 42, 9)
2021-07-26 13:16:05,585 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-26 13:16:05,585 INFO sqlalchemy.engine.Engine [cached since 0.001671s ago] (1, 42, 2)
2021-07-26 13:16:05,586 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-26 13:16:05,586 INFO sqlalchemy.engine.Engine [cached since 0.002639s ago] (1, 13, 2)
2021-07-26 13:16:05,587 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, user_id, qty) VALUES (?, ?, ?)
2021-07-26 13:16:05,587 INFO sqlalchemy.engine.Engine [cached since 0.00366s ago] (2, 13, 3)
2021-07-26 13:16:05,588 INFO sqlalchemy.engine.Engine INSERT INTO "order" (product_id, 

---

This is how we can pull all `User`s first names from the database:

In [41]:
session.query(User.first_name).all()

2021-07-26 13:05:42,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-26 13:05:42,665 INFO sqlalchemy.engine.Engine SELECT user.first_name AS user_first_name 
FROM user
2021-07-26 13:05:42,665 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()


[('Graham',), ('John',), ('Eric',), ('Terry',)]

That corresponds roughly to a `SELECT first_name FROM user` in SQL terms.

In [42]:
session.query(User.first_name).filter(User.id > 15).all()

2021-07-26 13:05:43,940 INFO sqlalchemy.engine.Engine SELECT user.first_name AS user_first_name 
FROM user 
WHERE user.id > ?
2021-07-26 13:05:43,941 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (15,)


[('John',), ('Eric',), ('Terry',)]

That corresponds roughly to a `SELECT first_name FROM user WHERE ...` in SQL terms.

---
## Exercise 1.

Taking the database from the example above, can you write ONE sqlalchemy query that will create an outer join on `Users`, `OrderItems` and `Products` that return the `last_name`, `qty` and `name` from the corresponding tables ?

The expected result should look like this:

2021-07-26 13:05:47,367 INFO sqlalchemy.engine.Engine SELECT user.last_name AS user_last_name, "order".qty AS order_qty, product.name AS product_name 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id LEFT OUTER JOIN product ON product.id = "order".product_id
2021-07-26 13:05:47,368 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ()


[('Chapman', 2, 'Rabbit'),
 ('Chapman', 3, 'Parrot'),
 ('Chapman', 20, 'Parrot'),
 ('Cleese', 2, 'Rabbit'),
 ('Cleese', 9, 'Rabbit'),
 ('Idle', None, None),
 ('Jones', None, None)]

---
## Exercise 2.

Taking the database from the example above, can you write ONE sqlalchemy query that will create an aggregated result from the query above (sum the products of qty and price accordingly) calculating the value of purchases (grouping) per `User` ? 

The expected result should look like this:

2021-07-26 13:10:05,561 INFO sqlalchemy.engine.Engine SELECT user.last_name AS user_last_name, sum("order".qty * product.price) AS sum_1 
FROM user LEFT OUTER JOIN "order" ON user.id = "order".user_id LEFT OUTER JOIN product ON product.id = "order".product_id GROUP BY user.id
2021-07-26 13:10:05,561 INFO sqlalchemy.engine.Engine [cached since 23.65s ago] ()


[('Chapman', Decimal('121.50')),
 ('Cleese', Decimal('23.10')),
 ('Idle', None),
 ('Jones', None)]